<a href="https://colab.research.google.com/github/AlejoAcelas/orcid2taxid/blob/master/HP_sciscpacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install scispacy spacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_core_sci_sm-0.5.4.tar.gz



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.3/243.3 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 13.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for en_core_sci_sm: filename=en_core_sci_sm-0.5.4-py3-none-any.whl size=14778487 sha256=9614f741a2eacfb946eae9346e12902c973de22c24a59ea9b1873b9ae3870a64
  Stored in directory: /root/.cache/pip/wheels/7f/29/44/dd461872b8547b8e8007f03418fb8061f5c05c71447982bcff
Successfully built en_core_sci_sm


In [4]:
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_ner_bionlp13cg_md-0.5.4.tar.gz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for en_ner_bionlp13cg_md: filename=en_ner_bionlp13cg_md-0.5.4-py3-none-any.whl size=119814670 sha256=de73429e9a4466bf70b000352d18ae68bbddb1329f31f9a0ec7fe4cdfc2c2829
  Stored in directory: /root/.cache/pip/wheels/31/ac/d1/be19808ecef0503b68100dfa881150a04957c7f9a2c4a6d0b1
Successfully built en_ner_bionlp13cg_md


In [32]:
import spacy
from scispacy.abbreviation import AbbreviationDetector

nlp = spacy.load("en_ner_bionlp13cg_md")

# Add the abbreviation pipe to the spacy pipeline.
nlp.add_pipe("abbreviation_detector")

In [33]:


# Load the biomedical NER model
nlp = spacy.load("en_ner_bionlp13cg_md")

# Test text with viruses and bacteria
test_text = "COVID-19 is caused by SARS-CoV-2. Studies also show interactions with Escherichia coli and Staphylococcus aureus."

# Process text
doc = nlp(test_text)

# Extract entities
entities = [(ent.text, ent.label_) for ent in doc.ents]
print("Extracted Entities:", entities)


Extracted Entities: [('Escherichia coli', 'ORGANISM'), ('Staphylococcus aureus', 'GENE_OR_GENE_PRODUCT')]


In [37]:
import spacy
import requests
from Bio import Entrez
import time
import re


def clean_text(text):
    """Removes XML tags and extra spaces from text."""
    return re.sub(r"<[^>]+>", " ", text).strip()


# Load SciSpaCy model for biomedical entity recognition
print("Loading SciSpaCy model...")
nlp = spacy.load("en_ner_bionlp13cg_md")

def extract_entities(text):
    """Extract only species-related entities (including viruses) using SciSpaCy."""
    doc = nlp(text)
    organism_labels = {"SPECIES", "TAXON", "ORGANISM", "VIRUS"}  # Allowed labels
    entities = [ent.text for ent in doc.ents if ent.label_ in organism_labels]
    return entities

def get_papers_from_orcid(orcid: str):
    """Get papers for a researcher using their ORCID"""
    url = f"https://pub.orcid.org/v3.0/{orcid}/works"
    headers = {"Accept": "application/json"}

    response = requests.get(url, headers=headers)
    if not response.ok:
        print(f"Error fetching ORCID data: {response.status_code}")
        return []

    data = response.json()
    papers = []

    for work in data.get('group', []):
        work_summary = work['work-summary'][0]

        paper = {
            'title': work_summary.get('title', {}).get('title', {}).get('value', 'No title'),
            'year': work_summary.get('publication-date', {}).get('year', {}).get('value', ''),
            'doi': None
        }

        external_ids = work_summary.get('external-ids', {}).get('external-id', [])
        for ext_id in external_ids:
            if ext_id['external-id-type'] == 'doi':
                paper['doi'] = ext_id['external-id-value']

        papers.append(paper)

    return papers

def get_abstract_from_doi(doi):
    """Try to get abstract using Crossref"""
    try:
        response = requests.get(f"https://api.crossref.org/works/{doi}")
        if response.ok:
            data = response.json()
            return data['message'].get('abstract', '')
    except:
        return ''
    return ''

def get_taxid(organism_name):
    """Get TAXID for an organism name"""
    try:
        handle = Entrez.esearch(db="taxonomy", term=organism_name)
        record = Entrez.read(handle)
        handle.close()

        if record["Count"] != "0":
            taxid = record["IdList"][0]

            handle = Entrez.efetch(db="taxonomy", id=taxid)
            details = Entrez.read(handle)
            handle.close()

            if details:
                return {
                    'taxid': taxid,
                    'scientific_name': details[0].get('ScientificName', ''),
                    'rank': details[0].get('Rank', ''),
                    'division': details[0].get('Division', '')
                }
    except Exception as e:
        print(f"Error getting TAXID for {organism_name}: {str(e)}")
    return None





Loading SciSpaCy model...


In [ ]:
# Test entity extraction
test_text = "COVID-19 is caused by SARS-CoV-2. Studies also show interactions with E. coli and Staphylococcus aureus."
doc = nlp(test_text)
entities = [(ent.text, ent.label_) for ent in doc.ents]
print("Extracted Entities:", entities)

In [39]:
# Run analysis
orcid = "0000-0002-7115-407X"  # Replace with your ORCID
Entrez.email = "hanna.palya@warwick.ac.uk"  # Replace with your email

# Get papers
print(f"Fetching papers for ORCID: {orcid}")
papers = get_papers_from_orcid(orcid)
print(f"Found {len(papers)} papers")

# Store found organisms and their TAXIDs
found_organisms = {}

# Process each paper
for paper in papers:
    print(f"\nAnalyzing paper: {paper['title']}")

    # Check text content
    text = paper['title']
    if paper['doi']:
        abstract = get_abstract_from_doi(paper['doi'])
        if abstract:
            text += " " + abstract

    print(f"Text for extraction: {text}")  # Debugging print

    text = clean_text(text)  # Clean before passing to extraction
    # Extract entities using SciSpaCy
    entities = extract_entities(text)

    print(f"Entities found: {entities}")  # Debugging print

    for entity in entities:
        print(f"Found organism: {entity}")

    # Look up each entity in taxonomy database
    for entity in entities:
        print(f"Found organism: {entity}")
        if entity not in found_organisms:
            tax_info = get_taxid(entity)
            if tax_info:
                found_organisms[entity] = tax_info
                print(f"Found organism: {entity}")
                print(f"TAXID: {tax_info['taxid']}")
                print(f"Scientific name: {tax_info['scientific_name']}")
                print("-" * 40)
        time.sleep(0.5)  # Be nice to NCBI servers

print("\nSummary of all organisms found:")
for org_name, info in found_organisms.items():
    print(f"\nOrganism: {org_name}")
    print(f"TAXID: {info['taxid']}")
    print(f"Scientific name: {info['scientific_name']}")

Fetching papers for ORCID: 0000-0002-7115-407X
Found 54 papers

Analyzing paper: Specific human gene expression in response to infection is an effective marker for diagnosis of latent and active tuberculosis
Text for extraction: Specific human gene expression in response to infection is an effective marker for diagnosis of latent and active tuberculosis <jats:title>Abstract</jats:title><jats:p>RNA sequencing and microarray analysis revealed transcriptional markers expressed in whole blood can differentiate active pulmonary TB (ATB) from other respiratory diseases (ORDs), and latent TB infection (LTBI) from healthy controls (HC). Here we describe a streamlined reverse transcriptase quantitative polymerase chain reaction (RT-qPCR) assay that could be applied at near point-of-care for diagnosing and distinguishing ATB from ORDs and LTBI from HC. A literature review was undertaken to identify the most plausible host-gene markers (HGM) of TB infection. Primers, and dual labelled hydrolysis 